## MicroPython on the ESP32

### Using Jupyter to interact with a ESP32 over its serial REPL

The following instructions assume that Jupyter is already installed, 
preferentially [Anaconda](https://www.anaconda.com/distribution/). This is a summary 
of the instructions in the respective github repository 
[goatchurchprime/jupyter_micropython_kernel](https://github.com/goatchurchprime/jupyter_micropython_kernel/)
by Julian Todd; please see there for further details and updates.

1. Clone this repository to a directory (e.g. `MicroPython`) using:
   ```
   git clone https://github.com/goatchurchprime/jupyter_micropython_kernel.git
   ```


2. Install this library (in editable mode) into Python3 using:
   ```
   pip install -e jupyter_micropython_kernel
   ```
   This creates a small file pointing to this directory in the `python/../site-packages directory`, 
   and makes it possible to `git update` the library later as it gets improved. 
 
 
3. Install the kernel into jupyter itself using:
   ```
   python -m jupyter_micropython_kernel.install
   ```
   This creates the small file `.local/share/jupyter/kernels/micropython/kernel.json` 
   that jupyter uses to reference its kernels.


4. Now run Jupyter notebooks:
   ```
   jupyter notebook
   ```
   
   
5. All "magic" commands can be listed with `%lsmagic`. The most important
   one connects the notebook to the MicroPython REPL on the device:

In [2]:
%serialconnect

Connecting to --port=COM5 --baud=115200 
OSError: [Errno 2] ENOENT
MicroPython v1.9.4-512-g6f015d337 on 2018-09-11; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


### Running some tests

Switching red onboard LED on ...

In [3]:
import machine

pin = machine.Pin(13, machine.Pin.OUT)
pin.value(1)

... and off:

In [4]:
pin.value(0)

Modulate the LED sinusoidally:

In [5]:
import math
import time

# Initialise PWM for pin 13
#
pwm = machine.PWM(machine.Pin(13))

# Change the duty cycle (0..1023) sinusoidally, `nCycle` times
#
nSteps  = 50
nCycles = 10
for iC in range(nCycles):
    for iS in range(nSteps):
        pwm.duty(int((math.sin(iS/nSteps *2 *math.pi) +1) /2 *1023))
        time.sleep(0.01)

# Release pin        
#
pwm.deinit()